https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

In [1]:
import glob
import pandas as pd
import numpy as np
import json

In [2]:
annotations=glob.glob('data/OPP-115/annotations/*')

In [3]:
annotations_list=[]
for file in annotations:
    annotations_list.append(pd.read_csv(file, header=None))

In [4]:
annot=pd.concat(annotations_list, ignore_index=True)

In [5]:
annot.columns=['annotation_ID', 'batch_ID', 'annotator_ID', 'policy_ID', 'segment_ID','category_name',
            'attribute_value_pairs','date','policy_URL']

In [6]:
annot['dict']=annot.attribute_value_pairs.apply(lambda x: json.loads(x))

In [7]:
annot.head()

,annotation_ID,batch_ID,annotator_ID,policy_ID,segment_ID,category_name,attribute_value_pairs,date,policy_URL,dict
0,13160,test_category_labeling_highlight_fordham_aaaa,121,3828,0,Other,"{""Other Type"": {""endIndexInSegment"": 575, ""sta...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Other Type': {'endIndexInSegment': 575, 'sta..."
1,13161,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
2,13162,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
3,13163,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
4,13164,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."


In [9]:
annot.iloc[307]['attribute_value_pairs']

'{"Collection Mode": {"selectedText": "personal information you supply when you subscribe, order, complete a form or survey, register for one of our Web Sites, enter a contest or provide your email address and", "startIndexInSegment": 100, "endIndexInSegment": 270, "value": "Explicit"}, "Choice Scope": {"selectedText": "null", "startIndexInSegment": -1, "endIndexInSegment": -1, "value": "not-selected"}, "Action First-Party": {"selectedText": "When you interact with SIDEARM Services, we collect:", "startIndexInSegment": 43, "endIndexInSegment": 95, "value": "Unspecified"}, "Personal Information Type": {"selectedText": "personal information", "startIndexInSegment": 100, "endIndexInSegment": 120, "value": "Generic personal information"}, "Choice Type": {"selectedText": "null", "startIndexInSegment": -1, "endIndexInSegment": -1, "value": "not-selected"}, "Identifiability": {"selectedText": "null", "startIndexInSegment": -1, "endIndexInSegment": -1, "value": "not-selected"}, "Does/Does Not"

In [10]:
for x in annot['dict']:
    for key, value in x.items():
        value.pop('endIndexInSegment', None)
        value.pop('startIndexInSegment', None)

In [11]:
text=[]
category=[]
subcat=[]
label=[]
counter=0
for i,x in enumerate(annot['dict']):
    for key, value in x.items():
        subcat.append(key)
        if value.get('selectedText')==None:
            text.append('noSelectedText')
        else:
            text.append(value.get('selectedText'))  
        category.append(annot['category_name'][i])
        for k, v in value.items():
            if k=='value':
                label.append(v)

In [12]:
d=list(zip(text, category, subcat, label))

In [13]:
data=pd.DataFrame(d, columns=['text', 'category', 'subcategory', 'label'])

In [14]:
data.head()

,text,category,subcategory,label
0,"Effective Date: May 7, 2015 Kraft Site Privacy...",Other,Other Type,Introductory/Generic
1,noSelectedText,First Party Collection/Use,Collection Mode,not-selected
2,collec,First Party Collection/Use,Choice Scope,Collection
3,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website
4,"personally-identifiable information, such as",First Party Collection/Use,Personal Information Type,Generic personal information


In [78]:
data['cat_sub'] = data['category'] +'-'+ data['subcategory']

In [84]:
data.groupby('cat_sub').count().shape

(36, 5)

In [15]:
data.groupby(['category','subcategory']).nunique().count()

text           36
category       36
subcategory    36
label          36
dtype: int64

In [16]:
data['cat_sub_lab'] = data['category'] +'-'+ data['subcategory'] +'-'+ data['label']

In [17]:
import string, re
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
data['text'] = data['text'].apply(lambda x : remove_punct(x.lower()))

data = data[data['text']!='null']
data = data[data['text']!='noselectedtext']
data = data[data['text']!='']
data = data[data['text']!=' ']
data.reset_index(inplace=True, drop=True)
ds = data[['text','cat_sub_lab']].drop_duplicates()

In [18]:
ds.shape

(36717, 2)

In [19]:
res = data['text'].apply(lambda x : re.split(' ', x.strip("[]")))
ds['tokenized_text']=res

In [20]:
ds[ds.text.str.contains('third party')].shape

(1024, 3)

In [21]:
from gensim.models import Word2Vec
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [22]:
w2v = Word2Vec(res)

2019-09-12 21:15:36,125 : INFO : collecting all words and their counts
2019-09-12 21:15:36,131 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-09-12 21:15:36,168 : INFO : PROGRESS: at sentence #10000, processed 96679 words, keeping 2033 word types
2019-09-12 21:15:36,194 : INFO : PROGRESS: at sentence #20000, processed 194289 words, keeping 2983 word types
2019-09-12 21:15:36,223 : INFO : PROGRESS: at sentence #30000, processed 294565 words, keeping 3819 word types
2019-09-12 21:15:36,270 : INFO : PROGRESS: at sentence #40000, processed 398030 words, keeping 4374 word types
2019-09-12 21:15:36,299 : INFO : PROGRESS: at sentence #50000, processed 503431 words, keeping 5052 word types
2019-09-12 21:15:36,336 : INFO : PROGRESS: at sentence #60000, processed 613515 words, keeping 5785 word types
2019-09-12 21:15:36,371 : INFO : PROGRESS: at sentence #70000, processed 723092 words, keeping 6193 word types
2019-09-12 21:15:36,403 : INFO : PROGRESS: at sentence

In [23]:
# summarize vocabulary
words = list(w2v.wv.vocab)
words.sort()

In [24]:
# access vector for one word
print(w2v['year'])

[-0.29724374  0.13251425  0.20087934 -1.1431371  -0.06104059  0.4888344
 -0.5342768   0.17300402  0.3978953  -0.32049942  0.9949632   0.4234791
 -0.80142087 -0.5654522   0.7478927   0.4596604   0.35013202  0.0805531
  1.123067    0.6542838  -0.06871755 -0.05286846 -0.5263083  -0.66176957
 -0.19429356  0.1397564   0.3167671  -0.5922285  -0.12346897  0.8873192
 -0.62962353  0.6721855   0.08176859 -0.34561294 -0.4758855  -0.4066681
  0.62534255  0.33512038  0.5254029  -0.6695289   0.42660487 -1.2495921
 -0.546845   -0.39895868  0.05962745 -0.6632697  -0.19186145 -0.08385835
  1.3668532  -0.0382739   0.17431995  0.2899158   0.86291033 -0.35177198
 -1.1461163   0.16577648 -0.37663215  0.19653952 -0.613202    0.7668813
  0.48367983 -0.17185394 -1.4808972   0.11437929 -0.47746623  0.48205647
  0.05510514 -0.43888164 -0.7445963  -0.39469555 -0.22051905  0.01838099
  0.20705195 -0.3203524  -0.49923754  0.5367914   0.66972095 -0.12687692
  0.98155034 -0.46355408 -1.1013707  -1.1577195  -0.01695 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [25]:
print(w2v.wv.most_similar(['first']))

2019-09-12 21:15:45,058 : INFO : precomputing L2-norms of word weight vectors


[('prior', 0.7148448824882507), ('real', 0.7077560424804688), ('student', 0.7053525447845459), ('screen', 0.6981539130210876), ('last', 0.6934001445770264), ('surname', 0.6887073516845703), ('name', 0.6856837272644043), ('class', 0.6825646162033081), ('daytime', 0.6693437695503235), ('picture', 0.6661283373832703)]


In [26]:
print(w2v.wv.most_similar(['collect']))

[('gather', 0.7927879095077515), ('collected', 0.604918897151947), ('nonpersonal', 0.6017380356788635), ('combine', 0.5860525369644165), ('store', 0.5835720300674438), ('collects', 0.5546083450317383), ('disclose', 0.5329210758209229), ('share', 0.5264827013015747), ('use', 0.5235240459442139), ('record', 0.5224952101707458)]


In [27]:
print(w2v.wv.most_similar(['rights']))

[('defend', 0.8219736814498901), ('property', 0.7660650014877319), ('enforce', 0.7649328708648682), ('safety', 0.7399484515190125), ('interference', 0.7230215072631836), ('obligations', 0.7081145644187927), ('resolve', 0.7032036781311035), ('disputes', 0.6768276691436768), ('policies', 0.6613006591796875), ('legal', 0.6582742929458618)]


In [28]:
w2v.vector_size

100

In [29]:
w2v.wv.similar_by_vector('data', topn=10)

[('information', 0.6309879422187805),
 ('statistics', 0.46953698992729187),
 ('metrics', 0.4690093994140625),
 ('fsds', 0.4501069188117981),
 ('statistical', 0.4296190142631531),
 ('informatio', 0.42614781856536865),
 ('combined', 0.41525596380233765),
 ('indirect', 0.40737688541412354),
 ('broad', 0.4066990315914154),
 ('trends', 0.4044736623764038)]

In [30]:
onehotlabels = pd.DataFrame(ds['cat_sub_lab'])
onehotlabels.iloc[0]

cat_sub_lab    Other-Other Type-Introductory/Generic
Name: 0, dtype: object

In [31]:
onehotlabels.shape

(36717, 1)

In [32]:
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
onehotlabels = encode.fit_transform(onehotlabels).toarray()

In [33]:
len(onehotlabels)

36717

In [34]:
print(onehotlabels[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [35]:
encode.inverse_transform(onehotlabels)

array([['Other-Other Type-Introductory/Generic'],
       ['First Party Collection/Use-Choice Scope-Collection'],
       ['First Party Collection/Use-Action First-Party-Collect on website'],
       ...,
       ['International and Specific Audiences-Audience Type-Citizens from other countries'],
       ['Other-Other Type-Privacy contact information'],
       ['Other-Other Type-Privacy contact information']], dtype=object)

In [36]:
onehotlabels[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [37]:
onehotlabels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [38]:
onehotlabels[0]+onehotlabels[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [39]:
type(onehotlabels)

numpy.ndarray

In [40]:
ds['encoded_labels'] = list(onehotlabels)

In [41]:
len(ds['encoded_labels'][0] + ds['encoded_labels'][1])

259

In [89]:
flat_tmp[flat_tmp['text']=='not sell']['tokenized_text']

12550    [not, sell, not, sell]
Name: tokenized_text, dtype: object

In [43]:
ds['encoded_labels'][0] + ds['encoded_labels'][1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [44]:
tmp = ds[['text','encoded_labels','tokenized_text']]

In [45]:
flat_tmp = tmp.groupby('text').sum().reset_index(level=0)

In [46]:
flat_tmp.shape

(28815, 3)

In [47]:
flat_tmp.encoded_labels.apply(lambda x : (x>1).sum()).unique()

array([0])

In [48]:
flat_tmp.encoded_labels.apply(lambda x : (x>1).sum()).sum()

0

In [49]:
len(flat_tmp)

28815

In [50]:
len(tmp)

36717

In [51]:
word_vectors = w2v.wv

In [52]:
word2id = {k: v.index for k, v in word_vectors.vocab.items()}

In [53]:
def add_new_word(new_word, new_vector, new_index, embedding_matrix, word2id):
    embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis=0)
    
    word2id = {word: (index+1) if index >= new_index else index for word, index in word2id.items()}
    word2id[new_word] = new_index
    return embedding_matrix, word2id

UNK_INDEX = 0
UNK_TOKEN = 'UNK'

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector, UNK_INDEX, embedding_matrix, word2id)

In [54]:
len(word2id)

4443

In [55]:
embedding_matrix.shape

(4443, 100)

In [56]:
embedding_matrix[0].shape

(100,)

In [57]:
embedding_matrix = np.insert(embedding_matrix, len(embedding_matrix), [np.zeros((100,))], axis=0)

In [58]:
len(embedding_matrix)

4444

In [60]:
embedding_matrix[4442].shape

(100,)

In [61]:
def get_int_data(token_text, word2id):
    x = []
    unk_count = 0
    for item in token_text:
        temp=[]
        x.append(temp)
        for word in item:
            if word in word2id:
                temp.append(word2id.get(word))
            else:
                temp.append(UNK_INDEX)
                unk_count += 1
    print('Data created. Percentage of unknown words: %.3f' % (unk_count))
    return np.array(x)

x=get_int_data(flat_tmp.tokenized_text, word2id)
x.shape

Data created. Percentage of unknown words: 5319.000


(28815,)

In [62]:
flat_tmp['x']=x

In [63]:
flat_tmp['x']=flat_tmp['x'].apply(lambda x: np.array(x))

In [64]:
max_length=max(flat_tmp['x'].apply(lambda x: len(x)))

In [97]:
flat_tmp[flat_tmp['count']==3]
flat_tmp.iloc[248]['tokenized_text']

['a',
 'information',
 'already',
 'collected',
 'as',
 'part',
 'of',
 'a',
 'particular',
 'purchase',
 'in',
 'our',
 'transactions',
 'database',
 'which',
 'may',
 'be',
 'shared',
 'with',
 'third',
 'parties',
 'involved',
 'in',
 'the',
 'processing',
 'of',
 'financial',
 'transactions',
 'as',
 'discussed',
 'above',
 'or',
 'a',
 'information',
 'already',
 'collected',
 'as',
 'part',
 'of',
 'a',
 'particular',
 'purchase',
 'in',
 'our',
 'transactions',
 'database',
 'which',
 'may',
 'be',
 'shared',
 'with',
 'third',
 'parties',
 'involved',
 'in',
 'the',
 'processing',
 'of',
 'financial',
 'transactions',
 'as',
 'discussed',
 'above',
 'or',
 'a',
 'information',
 'already',
 'collected',
 'as',
 'part',
 'of',
 'a',
 'particular',
 'purchase',
 'in',
 'our',
 'transactions',
 'database',
 'which',
 'may',
 'be',
 'shared',
 'with',
 'third',
 'parties',
 'involved',
 'in',
 'the',
 'processing',
 'of',
 'financial',
 'transactions',
 'as',
 'discussed',
 'above',

In [ ]:
from keras.preprocessing.sequence import pad_sequences
padded_docs = pad_sequences(flat_tmp.x, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

In [ ]:
target=pad_sequences(flat_tmp.encoded_labels)

In [ ]:
padded_docs

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_docs, target, test_size=0.3, random_state = 42)

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Flatten

In [ ]:
vocab_length = len(embedding_matrix)
model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(270, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(x_train, y_train, epochs=25, batch_size=100, validation_data=(x_test, y_test))

In [ ]:
vocab_length = len(embedding_matrix)
model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(270, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001), metrics=['accuracy'])
history=model.fit(x_train, y_train, epochs=50, batch_size=100, validation_data=(x_test, y_test))

In [ ]:
y_train[12].shape

In [ ]:
target.shape

In [ ]:
print(model.evaluate(x_train, y_train))
print(model.evaluate(x_test, y_test))

In [ ]:
results_train=pd.DataFrame(data=model.predict_proba(x_train))
results_test=pd.DataFrame(data=model.predict_proba(x_test))

In [ ]:
np.where(y_train[3]==1)

In [ ]:
classes_train=model.predict_classes(x_train)
classes_test=model.predict_classes(x_test)

In [ ]:
classes_train[3]

In [ ]:
results_train.iloc[3].sort_values(ascending=False)

In [65]:
y=flat_tmp.encoded_labels.apply(lambda x: x.sum())

In [66]:
max(y)

32.0

In [67]:
y[y==32]

9736    32.0
Name: encoded_labels, dtype: float64

In [68]:
flat_tmp.iloc[9736]['encoded_labels']

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [69]:
flat_tmp.iloc[9736]['text']

'information'

In [ ]:
data[data['text']=='null']

In [70]:
temp=flat_tmp[flat_tmp['text']!='information']

In [72]:
z=temp.encoded_labels.apply(lambda x: x.sum())

In [73]:
z[z==30]

12549    30.0
Name: encoded_labels, dtype: float64

In [74]:
temp.iloc[12549]

text                                                       not sell
encoded_labels    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
tokenized_text                               [not, sell, not, sell]
x                                                  [7, 251, 7, 251]
Name: 12550, dtype: object

In [81]:
flat_tmp['count']=y

In [83]:
flat_tmp.groupby('count').count()

,text,encoded_labels,tokenized_text,x
count,,,,
1.0,23552,23552,23552,23552
2.0,3834,3834,3834,3834
3.0,935,935,935,935
4.0,259,259,259,259
5.0,99,99,99,99
6.0,56,56,56,56
7.0,24,24,24,24
8.0,16,16,16,16
9.0,10,10,10,10
